In [ ]:
# -*- coding: utf-8 -*-
"""
Colab Asset Validator

This notebook validates the key assets (.keras, .json, .csv) for the federated
learning server in a clean Colab environment. It serves as the ultimate
"source of truth" for the model's benchmark performance.
"""

# Install necessary packages quietly
!pip install joblib -q

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
import json
from google.colab import drive
from pathlib import Path

# --- Mount Google Drive ---
print("--> Mounting Google Drive...")
drive.mount('/content/drive')

# --- Configuration: Point to your files in Google Drive ---
# IMPORTANT: Adjust this base path to match YOUR Google Drive structure
# This should point to the folder containing your 'data' and 'model' subfolders.
# For example, if your folder is MyDrive/FL_Project/, set this path.
BASE_DRIVE_PATH = Path('/content/drive/MyDrive/Colab_HAR_Project/') #<-- CHANGE THIS

# --- Define Paths to Assets ---
DATA_DIR = BASE_DRIVE_PATH / 'data'
MODEL_DIR = BASE_DRIVE_PATH / 'results/TCN_Results_v2_with_Assets'

# Source files
INPUT_CSV_1 = DATA_DIR / 'resampled_normalized_phone_data.csv'
INPUT_CSV_2 = DATA_DIR / 'combined_collected_data.csv'
SCALER_PATH = MODEL_DIR / 'tcn_v2_ABCDE_scaler.json'
MODEL_PATH = MODEL_DIR / 'tcn_v2_ABCDE_har_model.keras'

# Parameters for consistency
WINDOW_SIZE = 60
STRIDE = 15
ALL_ACTIVITIES_TO_KEEP = sorted(['A', 'B', 'C', 'D', 'E'])

print("\n--> Configuration Complete. Paths set.")

--> Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--> Configuration Complete. Paths set.


In [ ]:
# REVISED CELL 2 for Colab_Asset_Validator.ipynb

print("--> Loading the definitive, pre-saved test set...")
try:
    test_set_dir = BASE_DRIVE_PATH / 'results' / 'TCN_Results_v2_with_Assets' / 'definitive_test_set'
    X_test_scaled = np.load(test_set_dir / 'X_test_scaled.npy')
    y_test = np.load(test_set_dir / 'y_test.npy')
    print("    - Definitive test set loaded successfully.")
    print(f"    - Shape of X_test_scaled: {X_test_scaled.shape}")
except Exception as e:
    print(f"FATAL ERROR: Could not load the definitive test set .npy files from {test_set_dir}")
    print(f"    - Please ensure you have run the modified training notebook first.")
    raise

--> Loading the definitive, pre-saved test set...
    - Definitive test set loaded successfully.
    - Shape of X_test_scaled: (14021, 60, 6)


In [ ]:
# REVISED CELL 3 for Colab_Asset_Validator.ipynb

print("--> Loading Keras model from Google Drive...")
# ... (model loading code is the same) ...
model = tf.keras.models.load_model(MODEL_PATH)

print("\n--> Evaluating model on the definitive test set...")
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)

# --- Final Report ---
print("\n" + "="*50)
print("       COLAB ASSET VALIDATION REPORT")
print("="*50)
print(f"Benchmark Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print("="*50)
if accuracy > 0.98:
    print("\nVERDICT: SUCCESS! The assets and the definitive test set are consistent.")
else:
    print("\nVERDICT: FAILURE! Something is still wrong.")

--> Loading Keras model from Google Drive...

--> Evaluating model on the definitive test set...

       COLAB ASSET VALIDATION REPORT
Benchmark Accuracy: 0.9965 (99.65%)

VERDICT: SUCCESS! The assets and the definitive test set are consistent.
